In [2]:
import pandas as pd 

In [3]:
review = pd.read_pickle('../output/bar_reviews_cleaned_and_tokenized.pickle')

## Now we must generate a dictionary which maps vocabulary into a number

In [ ]:

# Generate a dictionary that maps vocab to a number
%time id2word_wiki = gensim.corpora.Dictionary(doc_stream)
print(id2word_wiki)



In [ ]:
# Now vectorize the documents using Bag Of Words
%time bow = id2word_wiki.doc2bow(tokenize(doc))
print(bow)